# Setup and libraries

## Load the needed libraries

These are the libraries I will be using for this notebook

In [157]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import json

%matplotlib inline

# Write files

In [168]:
# This section produces the data for a generative model of the Lunar Lander
# Create a single dataframe with all the data
# each row is a single run of a single model
# each column is a single timestep of a single variable

# NOTE:  There needs to be some thinking here.  I mean, while the position,
# velocity, and angle are all continuous, the thrust is not.  So, we need to
# thinkg about how to interpolate the thrust. I think the data in this case
# needs to be "ragged" in the sense that each row has a different number of
# entries.  However, perhaps we can also just look at the "shortest" run and
# truncate all the other runs to that length. 


# TODO:  This is getting close, but is not there yet.  I want things like
# 'x,x,x' to be something like 'x1,x2,x3' so that I can use the autoencoder
# more easily.  Is that a matter of combining the columns?  I think so.  
# How about keeping a dict to map times to indices?  That would work I think.

def uniform_data_for_autoencoder(info, entries_per_run=100):
    all_data = []
    for model_name in info['models']:
        for run_idx in range(info['number_of_trajectories']):
            df = pd.read_parquet(f'data/lander/{model_name}_{run_idx}_trajectory.parquet')  
            # There is a nice way to resample the data in pandas, but it requires a datetime index
            df['timestamp'] = pd.to_datetime(df.index, unit='s')
            df.set_index('timestamp', inplace=True)

            # However, we just want the number of seconds since the start of the run
            # so we just keep that as a column
            df['time_seconds'] = (df.index - df.index[0]).total_seconds()

            # We now compute the delta t that gives us 100 total sample points for each run
            # We do this by taking the total time of the run and dividing by 100
            total_time = df.index[-1] - df.index[0]
            delta_t = total_time / entries_per_run
            df = df.resample(delta_t).interpolate()

            # Ok, now things are resampled and interpolated, but we need to get rid of the
            # datetime index and replace it with a simple integer index and the number of seconds.
            #multi_index = pd.MultiIndex.from_arrays([np.arange(len(df))], names=('idx',))
            df.index = np.arange(len(df))

            # Melt makes a mutli-column dataframe into a single column dataframe (well, actually
            # a pair of columns, one for the variable name and one for the value).  
            df_melt = pd.melt(df, 
                  value_vars=['x', 'y', 'vx', 'vy', 'theta', 'vtheta', 'time_seconds'], 
                  var_name='parameter', 
                  ignore_index=False, 
                  value_name=(model_name, run_idx),
                  col_level=0)

            # We now have a dataframe with a single column, but we want to make the index
            # better for later slicing.  In particular, we want to make the index a multi-index
            # with the first index being the row number and the second index being the parameter
            # name.  This will make it easy to slice out all the x values, for example.
            df_melt.index = pd.MultiIndex.from_arrays([df_melt.index, df_melt['parameter']],names=('idx', 'parameter'))
            df_melt.drop(columns=['parameter'], inplace=True)

            # We now have a dataframe with a single column, but we want each experiment to be
            # a single row.  
            experiment = df_melt.T

            # Last but not least we want to add the model name and run index to the dataframe
            experiment.index = pd.MultiIndex.from_tuples(experiment.index, names=('run_idx', 'experiment'))
            
            all_data.append(experiment)
    all_data = pd.concat(all_data)
    return all_data
info = json.load(open('data/lander/info.json', 'r'))
all_data = uniform_data_for_autoencoder(info)

In [174]:
all_data.to_parquet('data/lander/all_data.parquet')

In [170]:
# Example of slicing out x,y values for time stepss 1..4 for all the runs of all the models
all_data.loc[:, (range(1,5),('x','y'))]

idx                        1                   2                   3  \
parameter                  x         y         x         y         x   
run_idx experiment                                                     
random  0          -0.012747  1.393964 -0.018426  1.385643 -0.024104   
        1           0.006329  1.401989  0.008917  1.397606  0.011504   
        2           0.000228  1.400376  0.000316  1.395521  0.000404   
        3          -0.009627  1.393494 -0.013567  1.383633 -0.017507   
        4          -0.012488  1.406938 -0.017625  1.394574 -0.022763   
...                      ...       ...       ...       ...       ...   
better  11         -0.004091  1.401998 -0.004021  1.387828 -0.003952   
        12         -0.004860  1.398826 -0.004690  1.384684 -0.004520   
        13          0.005949  1.387995  0.006160  1.373964  0.006370   
        14          0.019459  1.427793  0.032523  1.436952  0.045586   
        15          0.005859  1.388904  0.009276  1.368832  0.012692   

idx                                  4            
parameter                  y         x         y  
run_idx experiment                                
random  0           1.377323 -0.029783  1.369002  
        1           1.393223  0.014092  1.388841  
        2           1.390667  0.000492  1.385813  
        3           1.373771 -0.021448  1.363909  
        4           1.382211 -0.027901  1.369847  
...                      ...       ...       ...  
better  11          1.373659 -0.003882  1.359489  
        12          1.370542 -0.004350  1.356400  
        13          1.359933  0.006581  1.345902  
        14          1.446111  0.058650  1.455270  
        15          1.348760  0.016109  1.328689  

[64 rows x 8 columns]

In [176]:
#  Just the better runs, but all the x values
all_data.loc[('better', slice(None)), (slice(None),('x',))]

idx                      0         1         2         3         4    \
parameter                  x         x         x         x         x   
run_idx experiment                                                     
better  0           0.000082  0.000679  0.001276  0.001872  0.002469   
        1           0.007717  0.026439  0.045162  0.062855  0.080548   
        2          -0.005465 -0.005837 -0.006208 -0.006580 -0.006952   
        3           0.003687  0.010454  0.017221  0.023988  0.030755   
        4           0.002357  0.003117  0.003877  0.004637  0.005397   
        5           0.001581  0.002536  0.003492  0.004447  0.005403   
        6          -0.005896 -0.006738 -0.007579 -0.008421 -0.009262   
        7           0.000068 -0.000168 -0.000404 -0.000641 -0.000877   
        8           0.003535  0.004304  0.005073  0.005842  0.006611   
        9           0.006468  0.006675  0.006881  0.007088  0.007294   
        10          0.006360  0.006368  0.006376  0.006383  0.006391   
        11         -0.004160 -0.004091 -0.004021 -0.003952 -0.003882   
        12         -0.005031 -0.004860 -0.004690 -0.004520 -0.004350   
        13          0.005739  0.005949  0.006160  0.006370  0.006581   
        14          0.006396  0.019459  0.032523  0.045586  0.058650   
        15          0.002443  0.005859  0.009276  0.012692  0.016109   

idx                      5         6         7         8         9    ...  \
parameter                  x         x         x         x         x  ...   
run_idx experiment                                                    ...   
better  0           0.003066  0.003662  0.004259  0.004856  0.005452  ...   
        1           0.097449  0.114350  0.130413  0.146477  0.160684  ...   
        2          -0.007324 -0.007695 -0.008067 -0.008439 -0.008810  ...   
        3           0.037522  0.044289  0.051055  0.057822  0.064589  ...   
        4           0.006157  0.006917  0.007677  0.008437  0.009197  ...   
        5           0.006358  0.007314  0.008269  0.009225  0.010180  ...   
        6          -0.010104 -0.010945 -0.011787 -0.012628 -0.013470  ...   
        7          -0.001114 -0.001350 -0.001587 -0.001823 -0.002060  ...   
        8           0.007380  0.008149  0.008918  0.009687  0.010456  ...   
        9           0.007501  0.007707  0.007914  0.008120  0.008327  ...   
        10          0.006399  0.006406  0.006414  0.006422  0.006429  ...   
        11         -0.003813 -0.003743 -0.003674 -0.003604 -0.003535  ...   
        12         -0.004180 -0.004009 -0.003839 -0.003669 -0.003499  ...   
        13          0.006791  0.007002  0.007212  0.007423  0.007633  ...   
        14          0.071713  0.083949  0.096185  0.108421  0.120658  ...   
        15          0.019525  0.022942  0.026358  0.029775  0.033191  ...   

idx                      91        92        93        94        95   \
parameter                  x         x         x         x         x   
run_idx experiment                                                     
better  0           0.014534  0.014159  0.013784  0.013409  0.013033   
        1          -0.087560 -0.087570 -0.087571 -0.087571 -0.087571   
        2          -0.039288 -0.039659 -0.040031 -0.040403 -0.040774   
        3           0.056008  0.056008  0.056008  0.056008  0.056008   
        4           0.071524  0.072284  0.073044  0.073804  0.074564   
        5           0.018012  0.017247  0.016483  0.015718  0.014954   
        6          -0.082470 -0.083311 -0.084152 -0.084994 -0.085835   
        7          -0.021448 -0.021685 -0.021921 -0.022158 -0.022394   
        8           0.004291  0.003372  0.002452  0.001533  0.000614   
        9           0.025261  0.025467  0.025674  0.025880  0.026087   
        10          0.007059  0.007067  0.007074  0.007082  0.007090   
        11          0.002164  0.002234  0.002303  0.002373  0.002442   
        12          0.010458  0.010628  0.010798  0.010968  0.011139   
        13     